In [38]:
import traceback
import urllib3
import xmltodict
import pandas as pd
from math import radians, cos, sin, asin, sqrt

In [39]:

def distance(lat1, lat2, lon1, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result
    return(c * r)

def getxml():
    url = "https://www.fuelwatch.wa.gov.au/fuelwatch/fuelWatchRSS?"
    http = urllib3.PoolManager()

    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

    response = http.request('GET', url, headers={'User-Agent':user_agent,})

    try:
        data = xmltodict.parse(response.data)
    except:
        print("Failed to parse xml from response (%s)" % traceback.format_exc())
    return data

In [49]:
data = getxml()

In [57]:
df = pd.DataFrame(data['rss']['channel']['item'])
df['latitude'] = df['latitude'].astype('double')
df['longitude'] = df['longitude'].astype('double')

In [80]:
CURRENT_LAT = -31.9817967
CURRENT_LON = 115.8109769
df['distance_from_me'] = df.apply(lambda row: distance(CURRENT_LAT, row['latitude'], CURRENT_LON, row['longitude']), axis=1)

In [81]:
MAX_DISTANCE = 10

In [82]:
# ,
stations = df[['title', 'description', 'price', 'distance_from_me']]
stations[stations['distance_from_me'] < MAX_DISTANCE].sort_values(by='price')

,title,description,price,distance_from_me
7,189.9: BP Connect Myaree,"Address: 246 Leach Hwy, MYAREE, Phone: (08) 93...",189.9,7.171886
75,193.9: BP Connect Wembley,"Address: 240 Cambridge St, WEMBLEY, Phone: (08...",193.9,4.666719
67,193.9: BP Highgate,"Address: 342 Beaufort St, PERTH, Phone: 0450 7...",193.9,6.879693
63,193.9: BP Connect North Perth,"Address: 349 Charles St, NORTH PERTH, Phone: (...",193.9,7.244162
62,193.9: Ampol Rous Head,"Address: Cnr North Mole Dr & Rous Head Rd, NOR...",193.9,9.476676
...,...,...,...,...
365,199.9: Ampol Willetton,"Address: 169 High Rd, WILLETTON, Phone: (08) 9...",199.9,9.385994
370,201.5: Vibe Subiaco,"Address: 123 Thomas St, SUBIACO, Phone: (08) 9...",201.5,3.791287
371,201.7: Puma Burswood,"Address: 265 Great Eastern Hwy, BURSWOOD, Phon...",201.7,8.902561
376,201.9: 7-Eleven Booragoon,"Address: 492 Marmion St, BOORAGOON, Phone: (08...",201.9,6.517994


In [83]:
df

,title,description,brand,date,price,trading-name,location,address,phone,latitude,longitude,site-features,distance_from_me
0,187.9: Atlas Fuel Ascot,"Address: 472 Great Eastern Hwy, ASCOT, Open Mo...",Atlas,2022-07-09,187.9,Atlas Fuel Ascot,ASCOT,472 Great Eastern Hwy,None,-31.925213,115.951987,", Open Mon: 04:00-22:00, Tue: 04:00-22:00, Wed...",14.716590
1,187.9: BP Redcliffe,"Address: 419 Great Eastern Hwy, REDCLIFFE, Pho...",BP,2022-07-09,187.9,BP Redcliffe,REDCLIFFE,419 Great Eastern Hwy,(08) 9277 9755,-31.930469,115.944728,", Open 24 hours",13.849283
2,189.5: NightOwl Shell Maddington,"Address: 1 Davison St, MADDINGTON, Phone: 0457...",Shell,2022-07-09,189.5,NightOwl Shell Maddington,MADDINGTON,1 Davison St,0457 884 662,-32.045398,115.988733,", Open Mon-Fri: 05:00-23:59, Sat: 05:00-23:59,...",18.190704
3,189.8: Atlas Fuel Kwinana,"Address: 1 Mandurah Rd, KWINANA BEACH, Phone: ...",Atlas,2022-07-09,189.8,Atlas Fuel Kwinana,KWINANA BEACH,1 Mandurah Rd,(08) 6377 7665,-32.235848,115.780215,ATM Air Bottled AdBlue Bottled Gas EFTPOS Fuel...,28.397409
4,189.9: BP Ellenbrook,"Address: 180 The Promenade, ELLENBROOK, Phone:...",BP,2022-07-09,189.9,BP Ellenbrook,ELLENBROOK,180 The Promenade,(08) 9277 4758,-31.775436,115.963073,", Open 24 hours",27.069920
...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,209.9: Herne Hill General Store,"Address: 777 Great Northern Hwy, HERNE HILL, P...",Puma,2022-07-09,209.9,Herne Hill General Store,HERNE HILL,777 Great Northern Hwy,(08) 9296 0771,-31.825036,116.016716,", Open Mon-Fri: 06:00-18:00, Sat: 06:00-18:00,...",26.096492
402,209.9: Karnup Roadhouse,"Address: 2263 Mandurah Rd, KARNUP, Phone: 0476...",BP,2022-07-09,209.9,Karnup Roadhouse,KARNUP,2263 Mandurah Rd,0476 913 075,-32.427429,115.774670,", Open 24 hours",49.669658
403,209.9: Hills Emporio,"Address: 1615 Canning Rd, KARRAGULLEN, Phone: ...",Independent,2022-07-09,209.9,Hills Emporio,KARRAGULLEN,1615 Canning Rd,0437 708 788,-32.086273,116.115545,", Open Mon: 07:00-18:00, Tue: 07:00-18:00, Wed...",30.971045
404,211.9: BP Baskerville,"Address: 1084 Great Northern Hwy, BASKERVILLE,...",BP,2022-07-09,211.9,BP Baskerville,BASKERVILLE,1084 Great Northern Hwy,(08) 9296 4229,-31.797752,116.021245,"Conditional Driveway Service,",28.511445
